This notebook is for Segmenting and Clustering Toronto

Import all necessary libaries

In [133]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Saved the wiki table as csv file,uploaded the csv to my github, used the link to download it to jupyterlab

Filter out "Not assigned" Boroughs

In [148]:
#download the table from my github
filename = "https://gist.githubusercontent.com/dioo94/19fc42019b168a80e4f65ed336cb8610/raw/13f19891e28bc67ed4293b2c6ea4470b1bc297a4/toronto_wiki.csv"
#load the csv file
df = pd.read_csv(filename)
df.head()
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[~df['Borough'].isin(['Not assigned'])]
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


set "not assigned" neighbourhood to be name of borough

In [149]:
#set "not assigned" neighbourhood to be name of borough
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


combine neighbourhoods with same postcode and borough

In [150]:
#combine neighbourhoods with same postcode and borough
df=df.groupby(['Postcode','Borough'], sort = False).agg(lambda x: ','.join(x)).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


print the number of rows of dataframe

In [151]:
df.shape

(103, 3)

Saved the coordinates table as csv file,uploaded the csv to my github, used the link to download it to jupyterlab

In [152]:
#download the table from my github
filename = "https://raw.githubusercontent.com/dioo94/Coursera_Capstone/master/Geospatial_Coordinates.csv"
#load the csv file
df_ll = pd.read_csv(filename)
df_ll = df_ll.rename(columns={'Postal Code': 'Postcode'})
df_ll.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [153]:
result=pd.merge(df,df_ll,on='Postcode')
result.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
